In [ ]:
#import the dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import pandas as pd

In [ ]:
# Load pre-trained model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('my_model_directory')
model = AutoModelForSeq2SeqLM.from_pretrained('my_model_directory')

In [ ]:

train_dataset= pd.read_csv('data.csv')

In [ ]:
# Define fine-tuning function
def fine_tune(model, train_dataset):
    summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

    # Prepare dataset for fine-tuning
    train_dataset = [{'article': row['text'], 'summary': row['summary']} for _, row in train_dataset.iterrows()]
    # Fine-tune the model
    model = summarizer.model
    model.train()
    for epoch in range(3):
        for batch in train_dataset:
            model.generate(input_ids=tokenizer.encode(batch['article'], return_tensors='pt'), max_length=100, min_length=30)

    # Return fine-tuned model
    return summarizer.model

In [ ]:
# Fine-tune the model
fine_tuned_model = fine_tune(model, train_dataset)

In [ ]:
# Use the fine-tuned model for summarization
summarizer = pipeline("summarization", model=fine_tuned_model, tokenizer=tokenizer)

In [ ]:
text = "Nagpur Police received a call claiming 25 armed men were outside 'Taarak Mehta Ka Ooltah Chashmah' actor Dilip Joshi's house. It's the same call where caller warned about bombs at Mukesh Ambani, Amitabh Bachchan and Dharmendra's residences. The number belongs to a boy working in SIM card company. His number was spoofed and police are looking for the real caller."

summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

print(summary[0]['summary_text'])

In [ ]:
# Save the fine-tuned model and tokenizer
fine_tuned_model.save_pretrained('my_model_directory')
tokenizer.save_pretrained('my_model_directory')